In [10]:

import pandas as pd
import numpy as np
import random
eps = np.finfo(float).eps
from numpy import log2 as log
df = pd.read_csv("Cleaned_Andhra_dataset.csv")
sample_size = 30
test_indices = random.sample(population=(df.index.tolist()),k=sample_size)
train_data = df.drop(test_indices)
# the loc method  location of test_indices and returns the data in that index
test_data = df.loc[test_indices]
test_data_target_value =  df.loc[test_indices]
test_data = test_data.drop(['reslt'], axis = 1)
# print(test_data_target_value)
# print(test_data)
# df = train_data

# feature = ['age', 'weight1', 'history', 'HB', 'IFA', 'BP1', 'education', 'res', 'reslt']


# In[346]:


# this function finds the class entropy
def find_entropy(df):
   # print(df)
#     takes the last element in the dataset that is the label(target value)
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
#     unique function retruns an array of unique elements ie.array([0, 1], dtype=int64)
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
#         entrophy value returned is 0.8228368841492257 after two iterations
   # print(entropy)
    return entropy


# In[348]:


def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
   # print(attribute)
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
    entropy2 += -fraction2*entropy
    return abs(entropy2)


# In[349]:


def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        #print(key)
#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]


# In[350]:


def get_subtable(df, node,value):
    return df[df[node] == value].reset_index(drop=True)


# In[351]:


def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
#     print(Class)
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_winner(df)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['reslt'],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree
  


# In[2]:


tree = buildTree(df)


# In[3]:


#import pprint
#pprint.pprint(tree)


# In[365]:


def predict(inst,tree):
    #This function is used to predict for any input variable 
    
    #Recursively we go through the tree that we built earlier

    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction


# In[6]:


prediction_list = []
for i in range(len(test_data)):
#     takes row by row the test data!
#    print(test_data.iloc[i])
    prediction_list.append(predict(test_data.iloc[i],df))
#for j in prediction_list:
   # print(j)
    


# In[381]:


# Calculating Accuracy
total_ones = 0
for x in test_data_target_value['reslt']:
    if x == 1:
        total_ones += 1 
total_ones
sample_size

accuracy = total_ones/sample_size
accuracy *= 100
accuracy

73.33333333333333